In [ ]:
!pip install gdown

# Kolekcja ponad 100 tys. nagłówków z polskich portali informacyjnych z października-listopada 2020 r.
!rm news_pl.jsonl
!gdown https://drive.google.com/uc?id=1w8sx3Rs2W1fZgSnJmsKKph0oHueH0yww

rm: cannot remove 'news_pl.jsonl': No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1w8sx3Rs2W1fZgSnJmsKKph0oHueH0yww
To: /content/news_pl.jsonl
42.7MB [00:01, 24.8MB/s]


In [ ]:
# Wczytujemy dokumenty jako listę słowników. Każdy wiersz to jeden dokument.

import json 


def read_json_documents(jsonl):
    docs = []
    with open(jsonl,'r') as file:
        for line in file:
            if(len(line.strip())>0):
                docs.append(json.loads(line))
    return docs


headlines_pl = read_json_documents('news_pl.jsonl')
print(len(headlines_pl))
print(headlines_pl[0])


116056
{'id': '81d72ff3d226249c001dd3515a413a60', 'language': 'pl', 'source': 'wirtualnemedia.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Aleksandra Budka po 8 latach rozstaje się z radiową Trójką', 'url': 'https://www.wirtualnemedia.pl/artykul/aleksandra-budka-po-8-latach-rozstaje-sie-z-radiowa-trojka', 'category': ['news']}


In [ ]:
# 500 tys. artykułów z polskiej Wikipedii
!rm wiki_pl.jsonl.gz
!rm wiki_pl.jsonl

!gdown https://drive.google.com/uc?id=1go0J35NtymB-cdogrLM3YyTsyCMCMPw0
!gunzip wiki_pl.jsonl.gz

wiki_docs = read_json_documents('wiki_pl.jsonl')

print(len(wiki_docs))
print(wiki_docs[0])

rm: cannot remove 'wiki_pl.jsonl.gz': No such file or directory
rm: cannot remove 'wiki_pl.jsonl': No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1go0J35NtymB-cdogrLM3YyTsyCMCMPw0
To: /content/wiki_pl.jsonl.gz
25.5MB [00:00, 33.3MB/s]
500000
{'id': '931674', 'title': 'Garnizony Armii Radzieckiej na terytorium Polski', 'category': ['Północna_Grupa_Wojsk_Armii_Radzieckiej', 'Garnizony_wojska_rosyjskiego', 'III_Rzeczpospolita', 'Stosunki_polsko-sowieckie', 'Stosunki_polsko-rosyjskie'], 'language': 'pl', 'published': '2016-01-01T02:57:20Z', 'url': 'https://pl.wikipedia.org/wiki/Garnizony Armii Radzieckiej na terytorium Polski'}


In [ ]:
# Przeszukiwanie sekwencyjne 'brute-force'

def bf_search(documents, strings):
    matching_docs = []
    for doc in documents:
        for s in strings:
            if s in doc['title']:
                matching_docs.append(doc)
    print("\n---------------------------\nTotal documents found: {}\n---------------------------\n"
          .format(len(matching_docs)))
            
    return matching_docs
def print_docs(documents, fields):

    for di, document in enumerate(documents):
        print("{}. {}\t{}".format(di+1, *([document[f] for f in fields])))

matching = bf_search(documents = headlines_pl,strings=['samolot'])
print_docs(matching[:5], ['id','title'])



---------------------------
Total documents found: 107
---------------------------

1. e50d4cc63e508bba38d9d125b3ded190	Wrocław: 36 hiszpańskich studentów z koronawirusem. Przylecieli jednym samolotem
2. ec4498474b4a4d0e893ece67045d9ed2	Chora na nowotwór wyruszyła na leczenie rządowym samolotem
3. 01e5965aa2a2a1da9925345eed4bd5df	18-letnia Julia poleciała do USA rządowym samolotem. "Jesteśmy z Tobą!"
4. 0843f4d8a46fc11f96c99c8b0af0b5a0	Koronawirus. 41-letnia Polka zatrzymana na lotnisku Schiphol. Gryzła i kopała załogę samolotu
5. f271eedb333a5f86f3a276afd3064e4c	Koniec latania "na wypasie". Przygotujcie się na cięcia kosztów i niewygody w samolotach


In [ ]:
# Lematyzacja dokumentów w SpaCy
# !pip install -U spacy
# !python -m spacy validate
# 
!pip install spacy-nightly --pre
!python -m spacy download pl_core_news_sm


import spacy
nlp_pl = spacy.load("pl_core_news_sm")

doc = nlp_pl(headlines_pl[0]['title'])
print([w.lemma_ for w in doc])




     |████████████████████████████████| 11.0MB 6.8MB/s 
     |████████████████████████████████| 245kB 39.0MB/s 
     |████████████████████████████████| 460kB 36.9MB/s 
     |████████████████████████████████| 8.7MB 19.7MB/s 
     |████████████████████████████████| 993kB 33.3MB/s 
     |████████████████████████████████| 122kB 41.4MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp36-none-any.whl size=107097 sha256=93a531adfc8733fb76d92935165e7287bb7a0121e0d4af95ec85353eab82b4d6
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=392413f4eda11d282b59bcea628c43af02c474a7c6dcba6bc7a2c64418f755d2
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built smart-open contextvars
ERROR: thinc 8.0.0rc3 has requireme

In [ ]:
# Najprostszy odwrócony indeks z lematyzacją


def get_inv_index(documents):
    index = {}
    matching_docs = []
    for d in documents:
        doc = nlp_pl(d['title'])
        for w in doc:
            if w.lemma_ in index:
                index[w.lemma_].append(d)
            else:
                index[w.lemma_]=[d]
        
    return index


inverted_index = get_inv_index(headlines_pl[:10])

for lemma in inverted_index:
    if len(inverted_index[lemma]) > 1:
        print("{} ({})\t{}".format(lemma, len(inverted_index[lemma]), [d['title'] for d in inverted_index[lemma]]))



z (2)	['Aleksandra Budka po 8 latach rozstaje się z radiową Trójką', 'Gabriela Łazarczyk odchodzi z "Gazety Wyborczej"']
. (2)	['Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'Xiaomi przedstawia smartfony Mi10T. Ekrany 6,67 cala i wydajne procesory w cenie od 249 do 649 euro']
i (3)	['Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'Xiaomi przedstawia smartfony Mi10T. Ekrany 6,67 cala i wydajne procesory w cenie od 249 do 649 euro', 'Trzy nowe kanały w Play Now i Play Now TV, w październiku otwarte okno']
medium (2)	['Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'Redaktorzy mediów niezależnych na Białorusi: trwa czystka przestrzeni medialnej']
nowy (2)	['Nowy serial „Król” od 6 listopada w Canal+ (oficjalny zwiastun)', 'Trzy nowe kanały w Play Now i Play Now TV, w październiku otwarte okno']
od (2)	['Nowy serial „Król” od 6 listopada

In [ ]:
# Wyszukiwanie w odwróconym indeksie

# Indeksowanie
inverted_index = get_inv_index(headlines_pl[:1000])
print("Distinct lemmas: {}.".format(len(inverted_index)))

# Normalizacja zapytania
user_query = 'media'
doc = nlp_pl(user_query)
print([w.lemma_ for w in doc])


# 'Wyszukiwanie' 
for lemma in [w.lemma_ for w in doc]:
    print(inverted_index[lemma])




Distinct lemmas: 3594.
['medium']
[{'id': 'a289227a9701294ee5cee628e7dd267e', 'language': 'pl', 'source': 'wirtualnemedia.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych', 'url': 'https://www.wirtualnemedia.pl/artykul/senat-odrzucil-sprawozdanie-krrit-za-2019-r-i-informacje-o-dzialalnosci-rady-mediow-narodowych', 'category': ['news']}, {'id': 'cff0c0a0641efbe21a0a55e9efc5c96d', 'language': 'pl', 'source': 'wirtualnemedia.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Redaktorzy mediów niezależnych na Białorusi: trwa czystka przestrzeni medialnej', 'url': 'https://www.wirtualnemedia.pl/artykul/redaktorzy-mediow-niezaleznych-na-bialorusi-trwa-czystka-przestrzeni-medialnej', 'category': ['news']}, {'id': 'abe71863651df31a9cc76f61834781a3', 'language': 'pl', 'source': 'newsweek.pl', 'published': '2020-10-01T00:00:00Z', 'title': 'Jarosław, śmielej! Prawicowe media wzywają obóz władzy do 

In [ ]:
# Odwrócony indeks nazw własnych?
doc = nlp_pl('Magazyn śledczy Anity Gargas')
print([w.lemma_ for w in doc.ents])

['Anita gargas']


## Apache Solr

## Instalacja w Colab

In [1]:
# Sprawdźmy wersję JAVA VM 

!java -version

openjdk version "11.0.9.1" 2020-11-04
OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [2]:
# Pobieramy katalog z konfiguracją indeksów news_pl i wiki_pl

!rm solr-preconf.tar.gz
!gdown https://drive.google.com/uc?id=1CUFilOS3raMmLkzEQeXW2PHxxZF2iusF
!tar -zxf solr-preconf.tar.gz


rm: cannot remove 'solr-preconf.tar.gz': No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1CUFilOS3raMmLkzEQeXW2PHxxZF2iusF
To: /content/solr-preconf.tar.gz
201MB [00:05, 40.0MB/s]


In [3]:
# Pobieramy i rozpakowujemy aktualną wersję Solra 

!rm -rf solr-8.7.0
!wget https://apache.mirrors.tworzy.net/lucene/solr/8.7.0/solr-8.7.0.tgz 
!tar -zxf  solr-8.7.0.tgz 


--2021-01-19 06:39:46--  https://apache.mirrors.tworzy.net/lucene/solr/8.7.0/solr-8.7.0.tgz
Resolving apache.mirrors.tworzy.net (apache.mirrors.tworzy.net)... 51.83.241.43
Connecting to apache.mirrors.tworzy.net (apache.mirrors.tworzy.net)|51.83.241.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200805960 (192M) [application/x-gzip]
Saving to: ‘solr-8.7.0.tgz’

solr-8.7.0.tgz      100%[===================>] 191.50M  11.4MB/s    in 26s     

2021-01-19 06:40:13 (7.37 MB/s) - ‘solr-8.7.0.tgz’ saved [200805960/200805960]



In [4]:
# Zatrzymujemy ew. działające instancje

!/content/solr-8.7.0/bin/solr stop -all
#!ps -ef | grep solr
#!kill -9 PID

# Startujemy instancję
!/content/solr-8.7.0/bin/solr start -force

# Kopiujemy konfigurację indeksów z folderów:

!/content/solr-8.7.0/bin/solr create_core  -force -c news_pl -d ./solr-preconf/server/solr/news_pl 
!curl 'http://localhost:8983/solr/admin/cores?action=RELOAD&core=news_pl'

!/content/solr-8.7.0/bin/solr create_core  -force -c wiki_pl -d ./solr-preconf/server/solr/wiki_pl 
!curl 'http://localhost:8983/solr/admin/cores?action=RELOAD&core=wiki_pl'

Waiting up to 180 seconds to see Solr running on port 8983 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]  
Started Solr server on port 8983 (pid=197). Happy searching!

    
Created new core 'news_pl'
{
  "responseHeader":{
    "status":0,
    "QTime":691}}

Created new core 'wiki_pl'
{
  "responseHeader":{
    "status":0,
    "QTime":448}}


In [5]:
# Sprawdźmy, czy utworzyły się indeksy

!curl http://localhost:8983/solr/admin/cores

# W wersji uruchamianej lokalnie oba indeksy powinny być również widoczne w konsoli: http://localhost:8983/solr/#/ 

{
  "responseHeader":{
    "status":0,
    "QTime":23},
  "initFailures":{},
  "status":{
    "news_pl":{
      "name":"news_pl",
      "instanceDir":"/content/solr-8.7.0/server/solr/news_pl",
      "dataDir":"/content/solr-8.7.0/server/solr/news_pl/data/",
      "config":"solrconfig.xml",
      "schema":"schema.xml",
      "startTime":"2021-01-19T06:40:34.426Z",
      "uptime":8425,
      "index":{
        "numDocs":0,
        "maxDoc":0,
        "deletedDocs":0,
        "indexHeapUsageBytes":0,
        "version":2,
        "segmentCount":0,
        "current":true,
        "hasDeletions":false,
        "directory":"org.apache.lucene.store.NRTCachingDirectory:NRTCachingDirectory(MMapDirectory@/content/solr-8.7.0/server/solr/news_pl/data/index lockFactory=org.apache.lucene.store.NativeFSLockFactory@1dd89a68; maxCacheMB=48.0 maxMergeSizeMB=4.0)",
        "segmentsFile":"segments_1",
        "segmentsFileSizeInBytes":69,
        "userData":{},
        "sizeInBytes":69,
        "size":"69 

### Użycie Solr

In [ ]:
# Metody do indeksowania dokumentów przez HTTP w uprzednio zdefiniowanym schemacie indeksu Solr

import json
import requests

def index_documents(solr_url, docs):
    bytes = json.dumps(docs).encode(encoding='UTF-8')
    if(len(docs)> 0):
        bytes = json.dumps(docs).encode(encoding='UTF-8')
        try:
            res = requests.post(url=solr_url+'/update/json/docs',
                                data=bytes,
                                headers={'Content-Type': 'application/octet-stream'})
            solr_url+'update?commit=true'
            print("Update response: {}".format(res))
        except Exception as e:
            print(str(e))

def index_by_batch(solr_url, docs, batch_size):
    print("USING CORE: {}" .format(solr_url))
    doc_buffer=[]
    for doci, doc in enumerate(docs):
        doc_buffer.append(doc)
        if len(doc_buffer)%batch_size==0:
            index_documents(solr_url = solr_url, docs=doc_buffer)
            print('Adding doc {} to index'.format(doci+1))
            doc_buffer.clear()
    print('Adding doc {} to index'.format(doci+1))
    index_documents(solr_url = solr_url, docs=doc_buffer)
    print('Done indexing.'.format(doci+1))
    
    requests.get(solr_url +'/update?commit=true')
 

        

In [ ]:
def delete_all_documents(solr_url):
    """
    Wywołanie równoważne z:
    !curl "http://localhost:8983/solr/news_pl/update?commit=true" -H "Content-Type: text/xml" --data-binary '<delete><query>*:*</query></delete>'
    """
    try:
        print(solr_url+'/update?commit=true')
        res = requests.get(url=solr_url+'/update?commit=true',
                                data='<delete><query>*:*</query></delete>'.encode(encoding='UTF-8'),
                                headers={'Content-Type': 'text/xml'})
        print(res)
    except Exception as e:
        print(str(e))

In [ ]:
# Przykłady indeksowania dokumentów    

doc1 = {'id':'136f7046b8c9bf01ba3d3fd331180b79', 
        'language':'pl', 
        'source':'dziennik.pl',
        'published':'2011-01-23T18:31:00Z',
        'category':['news'],
        'title':'Tajemnicza historia chińskiego niewidzialnego samolotu', 
        'url':'https://wiadomosci.dziennik.pl/swiat/artykuly/318888,tajemnicza-historia-chinskiego-niewidzialnego-samolotu.html'
       }

# Opcjonalnie możemy najpierw usunąć wszystkie dokumenty
print("Clearing the index...")

s_url = 'http://localhost:8983/solr/news_pl'
delete_all_documents(solr_url = s_url)

# Dodajemy testowo 1 dokument:
index_by_batch(solr_url = s_url, docs=[doc1], batch_size=10)


Clearing the index...
http://localhost:8983/solr/news_pl/update?commit=true
<Response [200]>
USING CORE: http://localhost:8983/solr/news_pl
Adding doc 1 to index
Update response: <Response [200]>
Done indexing.


In [ ]:
# Dodajemy ponad 100 tys. dokumentów

delete_all_documents(solr_url = s_url)
headlines_pl = read_json_documents('news_pl.jsonl')
print("Headlines to index {}.".format(len(headlines_pl)))

index_by_batch(solr_url=s_url, docs = headlines_pl, batch_size=10000)


http://localhost:8983/solr/news_pl/update?commit=true
<Response [200]>
Headlines to index 116056.
USING CORE: http://localhost:8983/solr/news_pl
Update response: <Response [200]>
Adding doc 10000 to index
Update response: <Response [200]>
Adding doc 20000 to index
Update response: <Response [200]>
Adding doc 30000 to index
Update response: <Response [200]>
Adding doc 40000 to index
Update response: <Response [200]>
Adding doc 50000 to index
Update response: <Response [200]>
Adding doc 60000 to index
Update response: <Response [200]>
Adding doc 70000 to index
Update response: <Response [200]>
Adding doc 80000 to index
Update response: <Response [200]>
Adding doc 90000 to index
Update response: <Response [200]>
Adding doc 100000 to index
Update response: <Response [200]>
Adding doc 110000 to index
Adding doc 116056 to index
Update response: <Response [200]>
Done indexing.


In [ ]:
#Przykłady odpytywania indeksu z poziomu Pythona 


from urllib.request import urlopen

import json
s_url = 'http://localhost:8983/solr/news_pl'

def print_response(response, fields=['id','title','score']):
    print("\n{}\nTotal documents found: {}\n{}\n"
          .format('-'*25,response['response']['numFound'],'-'*25))
    # Print the name of each document.
    for di, document in enumerate(response['response']['docs']):
        print("{}. {}\t{}\t{}".format(di+1, *([document[f] for f in fields])))
        
    if 'facet_counts' in response:
        print("\n FACETS: \n")
        print(response['facet_counts']['facet_fields'])


In [ ]:
# 1. Proste zapytanie o 10 dokumentów zawierających dowolną formę słowa 'samolot'
connection = urlopen(s_url+'/select?q=title:samolot&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


-------------------------
Total documents found: 128
-------------------------

1. wnp.pl	Czas na mniejsze samoloty?	4.300211
2. interia.pl	Stworzył własny samolot elektryczny	4.300211
3. wnp.pl	Nieszczepieni samolotem nie polecą	4.300211
4. tvp.info	Kraksa samolotów. Pięć osób zginęło	4.1164083
5. wnp.pl	Lufthansa uziemi jeszcze więcej samolotów	4.1164083
6. interia.pl	Niezidentyfikowany samolot zauważony nad Kalifornią	4.1164083
7. onet.pl	Awaryjne lądowanie samolotu AN-124	4.1164083
8. onet.pl	Awaryjne lądowanie samolotu AN-124	4.1164083
9. polsatnews.pl	Zderzenie samolotów we Francji. Są ofiary	3.9476738
10. tvp.info	Niemieccy mechanicy odmówili serwisu samolotu Łukaszenki	3.9476738


In [ ]:
# Składnia DisMax
# https://lucene.apache.org/solr/guide/8_7/the-dismax-query-parser.html

import urllib

# 1.Zapytanie o 10 dokumentów zawierających dowolną formę słowa 'samolot' lub 'lotniczy'
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('title:samolot OR title:lotniczy')
    +'&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


-------------------------
Total documents found: 235
-------------------------

1. gazeta.pl	Samoloty wracają nad Wisłę. Lotniczy pokaz dla upamiętnienia 102. rocznicy odzyskania niepodległości	6.3987446
2. polsatnews.pl	Najdłuższe połączenie lotnicze świata wraca do rozkładu lotów. Samoloty pokonają ponad 15 tys. km	6.003709
3. tvn24.pl	Do samolotu wejdą tylko zaszczepieni. To pierwsza linia lotnicza, która zdecydowała się na taki krok	5.823934
4. wp.pl	Szczepionka na COVID warunkiem wejścia na pokład samolotu? Pierwsza linia lotnicza zdecydowała się na taki krok	5.654613
5. dzienniklodzki.pl	Koronawirus. Szczepienie może być wymagane do lotu samolotem. Linia Qantas to pierwsza linia lotnicza, która zdecydowała się na taki krok	5.0655265
6. wnp.pl	Czas na mniejsze samoloty?	4.300211
7. interia.pl	Stworzył własny samolot elektryczny	4.300211
8. wnp.pl	Nieszczepieni samolotem nie polecą	4.300211
9. tvp.info	Kraksa samolotów. Pięć osób zginęło	4.1164083
10. wnp.pl	Lufthansa uziemi jeszc

In [ ]:
# Wzbogacanie indeksu o jednostki nazewnicze (Zob. Moduł 6.4)

import spacy
nlp_pl = spacy.load("pl_core_news_sm")

def annotate_documents(documents):
    for di, d in enumerate(documents):
        doc = nlp_pl(d["title"])
        named_ents=["{}___{}".format(w.lemma_, w.label_) for w in doc.ents]
        if len(named_ents) > 0:
           d["ents"]=named_ents
           d["ent_types"]=[w.label_ for w in doc.ents]
        if((di)%1000 == 0):
            print("Annotated {} of {} documents.".format(di+1,len(documents)))
    return documents

headlines_pl = read_json_documents('news_pl.jsonl')
print("Headlines to index {}.".format(len(headlines_pl)))

annotated_headlines = annotate_documents(headlines_pl[0:10])

for ah in annotated_headlines:
  if 'ents' in ah:
    print("{}\t{}\n".format(ah['ents'], ah['title']))

#s_url = 'http://localhost:8983/solr/news_pl'
#delete_all_documents(solr_url = s_url)

# Dodajemy testowo 1 dokument:
#index_by_batch(solr_url = s_url, docs=[doc1], batch_size=10)

Headlines to index 116056.
Annotated 1 of 10 documents.
['Aleksander Budek___persName', 'trójka___orgName']	Aleksandra Budka po 8 latach rozstaje się z radiową Trójką

['senat___orgName', 'krrit___orgName', '2019 r .___date', 'rada medium narodowy___orgName']	Senat odrzucił sprawozdanie KRRiT za 2019 r. i informację o działalności Rady Mediów Narodowych

['6 listopad___date', 'canal+___placeName']	Nowy serial „Król” od 6 listopada w Canal+ (oficjalny zwiastun)

['plusa___orgName']	Technologia 5G dostępna także dla abonentów Plusa na Kartę

['Białoruś___placeName']	Redaktorzy mediów niezależnych na Białorusi: trwa czystka przestrzeni medialnej

['Gabriel łazarczyk___persName', '" gazeta wyborczy "___orgName']	Gabriela Łazarczyk odchodzi z "Gazety Wyborczej"

['tomasz jankowski___persName', 'zarząd zasobu komunalnego w Wrocław___orgName']	Tomasz Jankowski rzecznikiem prasowym Zarządu Zasobu Komunalnego we Wrocławiu

['play now i play now tv___orgName', 'październik___date']	Trzy nowe kan

In [ ]:
# Reindeksujemy dokumenty

annotated_headlines = annotate_documents(headlines_pl[0:500000])

s_url = 'http://localhost:8983/solr/news_pl'

delete_all_documents(solr_url = s_url)
print("Headlines to index {}.".format(len(annotated_headlines)))

index_by_batch(solr_url=s_url, docs = annotated_headlines, batch_size=10000)


Annotated 1 of 116056 documents.
Annotated 1001 of 116056 documents.
Annotated 2001 of 116056 documents.
Annotated 3001 of 116056 documents.
Annotated 4001 of 116056 documents.
Annotated 5001 of 116056 documents.
Annotated 6001 of 116056 documents.
Annotated 7001 of 116056 documents.
Annotated 8001 of 116056 documents.
Annotated 9001 of 116056 documents.
Annotated 10001 of 116056 documents.
Annotated 11001 of 116056 documents.
Annotated 12001 of 116056 documents.
Annotated 13001 of 116056 documents.
Annotated 14001 of 116056 documents.
Annotated 15001 of 116056 documents.
Annotated 16001 of 116056 documents.
Annotated 17001 of 116056 documents.
Annotated 18001 of 116056 documents.
Annotated 19001 of 116056 documents.
Annotated 20001 of 116056 documents.
Annotated 21001 of 116056 documents.
Annotated 22001 of 116056 documents.
Annotated 23001 of 116056 documents.
Annotated 24001 of 116056 documents.
Annotated 25001 of 116056 documents.
Annotated 26001 of 116056 documents.
Annotated 2700

In [ ]:
# Zapytanie o tytuły zawierające nazwę miejsca

import urllib

# 1.Zapytanie o 10 dokumentów zawierających dowolną formę słowa 'samolot' a zarazem nazwę miejsca
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('title:samolot AND ent_types:placeName')
    +'&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


-------------------------
Total documents found: 41
-------------------------

1. polsatnews.pl	Zderzenie samolotów we Francji. Są ofiary	4.3692303
2. tvp.info	Niemieccy mechanicy odmówili serwisu samolotu Łukaszenki	4.3692303
3. naszdziennik.pl	Mazowieckie: Awaryjne lądowanie samolotu w Modlinie	4.3692303
4. wp.pl	Francja. Zderzenie dwóch małych samolotów. Są ofiary	4.213784
5. polsatnews.pl	Awantura na pokładzie samolotu. Polka pobiła współpasażerki	4.213784
6. interia.pl	Czeskie samoloty morskie dla polskiej Straży Granicznej	4.213784
7. interia.pl	Awaryjne lądowanie samolotu na lotnisku w Modlinie	4.213784
8. interia.pl	Czeskiej produkcji samoloty już strzegą polskich granic	4.213784
9. onet.pl	USA. Szamotanina na pokładzie samolotu. Poszło o maseczkę	4.070116
10. polsatnews.pl	Awaryjne lądowanie samolotu w Modlinie. Problemy z podwoziem	4.070116


In [ ]:
# Zapytanie o tytuły zawierające nazwę miejsca

import urllib

# 1.Zapytanie o 10 dokumentów zawierających wzmianki osób zaczynające się od ciągu 'Gabriel'
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('ents:Gabriel*___persName')
    +'&wt=json&rows=10&fl=*,score')
response = json.load(connection)
print_response(response=response, fields=['source','title','score'])


-------------------------
Total documents found: 14
-------------------------

1. wirtualnemedia.pl	Gabriela Łazarczyk odchodzi z "Gazety Wyborczej"	1.0
2. interia.pl	Gabriela Morawska-Stanecka o nowym ministrze edukacji: Skandaliczna nominacja	1.0
3. kobieta.wp.pl	Gabrielę pasjonują perfumy. Ma 250 flakonów i zdradza, jak dobrać idealny zapach	1.0
4. press.pl	Gabriela Dworniak przestała kierować Agencją Kreacji Rozrywki i Oprawy TVP	1.0
5. rp.pl	Gabriela Morawska-Stanecka: Wyrok TK wynika z tchórzostwa	1.0
6. interia.pl	Rafał i Gabriel z "Królowych życia" zostali okradzeni	1.0
7. plotek.pl	"Królowe życia". Rafał i Gabriel zostali okradzeni. "Nie mieliśmy jak wejść do domu"	1.0
8. interia.pl	Rafał i Gabriel z "Królowych życia" zostali okradzeni! Przeżyli dramat!	1.0
9. gazeta.pl	Aktywistka Gabriela Lazarek spędziła noc "na dołku". Wyszła z poważnymi zarzutami	1.0
10. plotek.pl	"Królowe życia". Dagmara Kaźmierska kpi ze związku Gabriela i Rafała: "To jest gorsze od tego, czym ja się za

In [ ]:
# Agregacja wszystkich pasujących dokumentów po typach nazw własnych

import urllib

# 1.Zapytanie o 5 dokumentów zawierających lemat 'Brazylia'
connection = urlopen(
    s_url+'/select?q='
    + urllib.request.quote('title:brazylia')
    +'&wt=json&rows=5&fl=*,score'
    +'&facet.field=ent_types&facet=on')
response = json.load(connection)
print_response(response=response, fields=['source','title','ents'])




-------------------------
Total documents found: 112
-------------------------

1. tvn24.pl	Ambasador odwołana z Brazylii	['Brazylia___placeName']
2. interia.pl	Brazylia: Znaczny spadek przyrostu zakażeń	['brazylia___persName']
3. rp.pl	Minister zdrowia Brazylii zakażony koronawirusem	['Brazylia___placeName']
4. tvn24.pl	Nie ma mocnych na Brazylię	['Brazylia___persName']
5. wnp.pl	Brazylia: 27 750 nowych przypadków koronawirusa	['brazylia___persName']

 FACETS: 

{'ent_types': ['persName', 82, 'placeName', 42, 'orgName', 7, 'geogName', 3, 'date', 0, 'time', 0]}
